In [1776]:
import pandas as pd
import sys
import numpy as np
import random
import math
import folium
from IPython.display import display
from folium.features import DivIcon
from numpy.random import randint, choice, permutation
from folium.plugins import BeautifyIcon

In [1777]:
data = 'sub_data_file.csv'
base_station_1 = (50.817205047239135, -1.4493866785949845) # south-east end (Beaulieu)
base_station_2 = (50.86236620061749, -1.5720659114815352) # north-west end (Lyndhurst)
x_y_base_station_1 = [151, (5000, -5000)]
x_y_base_station_2 = [152, (-5000, 5000)]
central_coordinate = ((base_station_1[0] + base_station_2[0]) / 2, (base_station_1[1] + base_station_2[1]) / 2)
distance_bandwidth_table = [
    [(3000, np.inf), 0],
    [(2500, 2999), 1],
    [(2000, 2499), 2],
    [(1500, 1999), 3],
    [(1000, 1499), 4],
    [(500, 999), 5],
    [(0, 499), 7]
]
node_to_node_latency = 30 #mbps
#map = folium.Map(location=central_coordinate, zoom_start=13, min_zoom=8,max_zoom=14, tiles="cartodb positron")
map = folium.Map(location=central_coordinate, control_scale=True, zoom_start=13, min_zoom=12,max_zoom=14) # init map

In [1778]:
def get_x_y_coordinates(data):
    return pd.read_csv(data, names=["id", "x", "y"])

In [1779]:
x_y_coords = get_x_y_coordinates(data)
#x_y_coords

In [1780]:
x_y_data = x_y_coords.values.tolist()
x_y_data.append([151, 5000, -5000])
x_y_data.append([152, -5000, 5000])
x_y_data = {int(data[0]): [data[1], data[2]] for data in x_y_data}
#x_y_data

In [1781]:
def calculate_distance(x_y_coords1, x_y_coords2):
    return math.sqrt((x_y_coords1[0] - x_y_coords2[0])**2 +\
                      (x_y_coords1[1] - x_y_coords2[1])**2)

In [1782]:
def convert_distance_bandwidth(distance):
    global distance_bandwidth_table
    for i in distance_bandwidth_table:
        if int(distance) >= i[0][0] and int(distance) <= i[0][1]:
            return i[1]

In [1783]:
def absolute_cost_function(end_to_end_bandwidth, end_to_end_latency, alpha=1, beta=150):
    return (end_to_end_bandwidth * alpha) + (1 / end_to_end_latency * beta)

In [1784]:
def get_geographic_coordinates(coords):
    global central_coordinate
    coordinates = {}
    for i in coords.iterrows():
        coordinates[str(int(i[1]["id"]))] = (i[1]["y"]/110574 + central_coordinate[0], i[1]["x"]/111320 * math.cos(central_coordinate[0]) + central_coordinate[1])
    node_no = len(coordinates)
    coordinates[str(node_no + 1)] = base_station_1
    coordinates[str(node_no + 2)] = base_station_2
    return coordinates

In [1785]:
geographic_coords = get_geographic_coordinates(x_y_coords)

In [1786]:
def init_blank_map():
    global base_station_1, base_station_2
    folium.Marker([base_station_1[0], base_station_1[1]],  popup='Base Station 1').add_to(map)
    folium.Marker([base_station_2[0], base_station_2[1]],  popup='Base Station 2').add_to(map)
    for location_id, location_info in geographic_coords.items():
        folium.CircleMarker([location_info[0], location_info[1]], radius=2, popup=f"Sensor ID: {location_id}").add_to(map)

In [1787]:
init_blank_map()

In [1788]:
def show_map():
    display(map)

In [1789]:
def save_map():
    map.save("map.html")

In [1790]:
# except for the first and last coordinates do duplicate all coordinates in the middle for the coordinates dict item
"""
coordinates = {'best_route': {'sensor_id': [19, 28, 145],
              'coordinates': [(50.83182308300549, -1.50844786872212),
               (50.821231533454956, -1.4924287442749806),
               (50.821231533454956, -1.4924287442749806),
               (50.80703108850407, -1.4877640416052273),
               (50.80703108850407, -1.4877640416052273),
               (50.817205047239135, -1.4493866785949845)]}, 
               'possible_routes': [[(50.79834911986768, -1.4912597403722938), (50.82619020366676, -1.5019504696113806),
                                    (50.82619020366676, -1.5019504696113806), (50.817205047239135, -1.4493866785949845)],
                                   [(50.79834911986768, -1.4912597403722938), (50.80721558033759, -1.5461448507014692),
                                    (50.80721558033759, -1.5461448507014692), (50.817205047239135, -1.4493866785949845)],
                                  ]
              }
"""

"\ncoordinates = {'best_route': {'sensor_id': [19, 28, 145],\n              'coordinates': [(50.83182308300549, -1.50844786872212),\n               (50.821231533454956, -1.4924287442749806),\n               (50.821231533454956, -1.4924287442749806),\n               (50.80703108850407, -1.4877640416052273),\n               (50.80703108850407, -1.4877640416052273),\n               (50.817205047239135, -1.4493866785949845)]}, \n               'possible_routes': [[(50.79834911986768, -1.4912597403722938), (50.82619020366676, -1.5019504696113806),\n                                    (50.82619020366676, -1.5019504696113806), (50.817205047239135, -1.4493866785949845)],\n                                   [(50.79834911986768, -1.4912597403722938), (50.80721558033759, -1.5461448507014692),\n                                    (50.80721558033759, -1.5461448507014692), (50.817205047239135, -1.4493866785949845)],\n                                  ]\n              }\n"

In [1791]:
def plot_route(coordinates):
    init_blank_map()
    # best route
    best_lines_group = folium.FeatureGroup(name="Best Route").add_to(map)
    best_lines_group.add_child(folium.PolyLine(locations=coordinates['best_route']['coordinates'], weight=3,color = 'yellow'))
    idx_increament = 0
    for idx, sensor in enumerate(coordinates['best_route']['sensor_id']):
        if idx == 0:
            maker_location = list(coordinates['best_route']['coordinates'][idx])
        else:
            maker_location = list(coordinates['best_route']['coordinates'][idx + idx_increament])
        #sensor_icon=folium.Icon(color='white', icon=str(sensor), icon_color="red", prefix='fa')
        sensor_icon = icon=BeautifyIcon(icon="arrow-down",icon_shape="marker",number=str(sensor),
                                        border_color= "yellow",background_color="#FFFFFF",text_color="red"
        )
        best_lines_group.add_child(folium.map.Marker(maker_location, icon=sensor_icon).add_to(map))
        idx_increament += 1

    # possible routes
    possible_lines_group = folium.FeatureGroup(name="Other Routes").add_to(map)
    for possible_line in coordinates['possible_routes']:
        line_colour = random.randrange(0, 2**24)
        hex_line_colour = hex(line_colour) 
        route_color = "#" + hex_line_colour[2:]
        line = folium.PolyLine(locations=possible_line, weight=2, color=route_color)
        possible_lines_group.add_child(line)

    folium.LayerControl().add_to(map)
    show_map()

In [1792]:
#plot_route(coordinates)